In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

from utils.functions import extract_tem_min, get_data_fromNSRDB, tabbed_df_viewer, plot_time_series

from utils.maestros import Maestro_Optimizacion

[    0.00] Initializing mpi-sppy


## Dimensionar MR OffGrid con escenarios de irradiancia y velocidad del viento

### Seleccionar ubicación

In [2]:
#location = {"lat":4.433, "lon":-69.8, "name":"Cumaribo", "name_data" : "cumaribo", "year_deterministic": 2020}
#location = {"lat":12.1755, "lon":-71.2831, "name":"nazareth", "name_data":"nazareth", "year_deterministic": 2020}
location = {"lat":5.1261, "lon":-70.8755, "name":"rosalia", "name_data":"rosalia", "year_deterministic": 2020}

### Escenario determinista

In [3]:
df_meteo, info = get_data_fromNSRDB(location["lat"], location["lon"], location["year_deterministic"])
date_vec = np.vectorize(datetime)
df_index = date_vec(df_meteo.Year.values,df_meteo.Month.values,df_meteo.Day.values, df_meteo.Hour.values, df_meteo.Minute.values, tzinfo=None)
df_meteo.index = df_index
df_meteo

,Year,Month,Day,Hour,Minute,GHI,DHI,DNI,Wind Speed,Temperature,Solar Zenith Angle
2020-01-01 00:00:00,2020,1,1,0,0,0,0,0,1.7,24.0,161.81
2020-01-01 01:00:00,2020,1,1,1,0,0,0,0,1.9,23.6,154.84
2020-01-01 02:00:00,2020,1,1,2,0,0,0,0,2.1,23.2,143.22
2020-01-01 03:00:00,2020,1,1,3,0,0,0,0,2.2,23.0,130.13
2020-01-01 04:00:00,2020,1,1,4,0,0,0,0,2.1,22.7,116.53
...,...,...,...,...,...,...,...,...,...,...,...
2020-12-31 19:00:00,2020,12,31,19,0,0,0,0,2.1,26.9,108.89
2020-12-31 20:00:00,2020,12,31,20,0,0,0,0,1.9,26.1,122.61
2020-12-31 21:00:00,2020,12,31,21,0,0,0,0,1.7,25.5,136.04
2020-12-31 22:00:00,2020,12,31,22,0,0,0,0,1.7,24.9,148.67


### Catálogo de equipos

In [4]:
catalogo = pd.read_excel("Catalogo.xlsx", sheet_name=None, index_col = 0)
tabbed_df_viewer(catalogo)

### Series temporales

In [5]:
series_data = pd.read_excel("CaseData.xlsx", sheet_name = f"series_{location['name_data']}")

plot_time_series(series_data,"T")

### Planteamiento de necesidades comunidad

In [6]:
needs = pd.read_excel("CaseData.xlsx", sheet_name = f"necesidades_{location['name_data']}", index_col = 0)

idx_hour_0 = needs.columns.get_loc(0)
needs_n = needs.iloc[:,:idx_hour_0]

needs_load = needs.iloc[:, idx_hour_0:].T.astype(float)
needs_load.index.name = "Hora"

visual_dfs = {"Eficiencias por necesidad" : needs_n, "Carga diaria nececidades (kW)" : needs_load}

needs_load = pd.concat([needs_load] * 365, ignore_index=True)

tabbed_df_viewer(visual_dfs)

### Parámetros del modelo

In [7]:
data_model = {}

# PARÁMETROS GEOGRÁFICOS
data_model["lat"] = location["lat"]
data_model["lon"] = location["lon"]

# CANTIDAD DE DATOS TEMPORALES
data_model["len_t"] = 8760

# PARÁMETROS ECONÓMICOS
data_model["interest"] = 0.1
data_model["lifeyears"] = 25

# PARÁMETROS NECESIDADES
data_model["needs"] = {}
data_model["needs"]["active"] = True
if data_model["needs"]["active"]:
    data_model["needs"]["n"] = needs_n
    data_model["needs"]["load"] = needs_load

# PARÁMETROS DE LA CARGA ELÉCTRICA
data_model["load_el"] = {}
data_model["load_el"]["active"] = True
if data_model["load_el"]["active"]:    
    data_model["load_el"]["type"] = "variable"
    data_model["load_el"]["value"] = series_data["LOAD_EL"].to_numpy()
    
    # PARÁMETROS DE LA ENERGIA ELÉCTRICA NO SUMINISTRADA
    data_model["load_el"]["ENS_EL"] = {}
    data_model["load_el"]["ENS_EL"]["active"] = True

    if data_model["load_el"]["ENS_EL"]["active"]:
        data_model["load_el"]["ENS_EL"]["type"] = "fixed"
        data_model["load_el"]["ENS_EL"]["value"] = 2

# PARÁMETROS DE LA CARGA TÉRMICA
data_model["load_th"] = {}
data_model["load_th"]["active"] = True
if data_model["load_th"]["active"]:    
    data_model["load_th"]["type"] = "variable"
    data_model["load_th"]["value"] = series_data["LOAD_TH"].to_numpy()

    # PARÁMETROS DE LA ENERGIA ELÉCTRICA NO SUMINISTRADA
    data_model["load_th"]["DES_TH"] = {}
    data_model["load_th"]["DES_TH"]["active"] = True

    if data_model["load_th"]["DES_TH"]["active"]:
        data_model["load_th"]["DES_TH"]["type"] = "fixed"
        data_model["load_th"]["DES_TH"]["value"] = 2
        data_model["load_th"]["DES_TH"]["porc"] = 0.1


# PARÁMETROS DE LA CARGA DE REFRIGERACIÓN
data_model["load_cl"] = {}
data_model["load_cl"]["active"] = True
if data_model["load_cl"]["active"]:    
    data_model["load_cl"]["type"] = "variable"
    data_model["load_cl"]["value"] = series_data["LOAD_CL"].to_numpy()

    # PARÁMETROS DE LA ENERGIA ELÉCTRICA NO SUMINISTRADA
    data_model["load_cl"]["DES_CL"] = {}
    data_model["load_cl"]["DES_CL"]["active"] = True

    if data_model["load_cl"]["DES_CL"]["active"]:
        data_model["load_cl"]["DES_CL"]["type"] = "fixed"
        data_model["load_cl"]["DES_CL"]["value"] = 2
        data_model["load_cl"]["DES_CL"]["porc"] = 0.1
    

# PARÁMETROS DE LOS INVERSORES
data_model["inverters"] = {}
data_model["inverters"]["active"] = True
if data_model["inverters"]["active"]:
    data_model["inverters"]["type"] = catalogo["Hybrid OnGrid"]
    data_model["inverters"]["flex"] = False


# PARÁMETROS DE LAS BATERÍAS
data_model["batteries"] = {}
data_model["batteries"]["active"] = True
if data_model["batteries"]["active"]:
    data_model["batteries"]["type"] = catalogo["BattModules"]
    data_model["batteries"]["soc_0"] = 0.5

# PARÁMETROS DE LOS PANELES SOLARES
data_model["pv_modules"] = {}
data_model["pv_modules"]["active"] = True
if data_model["pv_modules"]["active"]:

    data_model["pv_modules"]["type"] = catalogo["PVModules"]

    Temp_min = extract_tem_min(data_model["lat"], data_model["lon"])
    for k in data_model["pv_modules"]["type"].columns:
        data_model["pv_modules"]["type"].loc['Voc_max', k] = np.round(
            data_model["pv_modules"]["type"].loc['Voc_STC',k]*(1+(data_model["pv_modules"]["type"].loc['Tc_Voc',k]/100)*(Temp_min-25))
        ,2)

# PARÁMETROS DE LAS TURBINAS EÓLICAS
data_model["windgen"] = {}
data_model["windgen"]["active"] = True
if data_model["windgen"]["active"]:
    data_model["windgen"]["type"] = catalogo["WindTurbines"]

# PARÁMETROS LAS CALDERAS (BOILERS)
data_model["boilers"] = {}
data_model["boilers"]["active"] = True
if data_model["boilers"]["active"]:
    data_model["boilers"]["type"] = catalogo["Boilers"]

# PARÁMETROS LOS CHPs
data_model["chps"] = {}
data_model["chps"]["active"] = True
if data_model["chps"]["active"]:
    data_model["chps"]["type"] = catalogo["CHPs"]

# PARÁMETROS LOS ENFRIADORES DE ABSORCIÓN
data_model["abs_chillers"] = {}
data_model["abs_chillers"]["active"] = True
if data_model["abs_chillers"]["active"]:
    data_model["abs_chillers"]["type"] = catalogo["AbsorptionChillers"]

# PARÁMETROS LOS ENFRIADORES ELÉCTRICOS
data_model["el_chillers"] = {}
data_model["el_chillers"]["active"] = True
if data_model["el_chillers"]["active"]:
    data_model["el_chillers"]["type"] = catalogo["ElectricChillers"]

# PARÁMETROS LOS CALENTADORES ELÉCTRICOS
data_model["el_heaters"] = {}
data_model["el_heaters"]["active"] = True
if data_model["el_heaters"]["active"]:
    data_model["el_heaters"]["type"] = catalogo["ElectricHeaters"]

# PARÁMETROS LOS CALENTADORES ELÉCTRICOS
data_model["pv_thermal"] = {}
data_model["pv_thermal"]["active"] = True
if data_model["pv_thermal"]["active"]:
    data_model["pv_thermal"]["type"] = catalogo["PVThermal"]

# PARÁMETROS DEL ALMACENAMIENTO TÉRMICO
data_model["heat_storage"] = {}
data_model["heat_storage"]["active"] = True
if data_model["heat_storage"]["active"]:
    data_model["heat_storage"]["type"] = catalogo["HeatStorage"]
    data_model["heat_storage"]["soc_0"] = 0

# PARÁMETROS DE LA RED ELÉCTRICA
data_model["grid_el"] = {}
data_model["grid_el"]["active"] = True
if data_model["grid_el"]["active"]:
    data_model["grid_el"]["pmax_buy"] = 800
    data_model["grid_el"]["pmax_sell"] = 0

    data_model["grid_el"]["buy_price"] = {}
    data_model["grid_el"]["buy_price"]["type"] = "variable"
    data_model["grid_el"]["buy_price"]["value"] = series_data["C_BUY_EL"].to_numpy()

    data_model["grid_el"]["sell_price"] = {}
    data_model["grid_el"]["sell_price"]["type"] = "variable"
    data_model["grid_el"]["sell_price"]["value"] = series_data["C_SELL_EL"].to_numpy()

    data_model["grid_el"]["av"] = {}
    data_model["grid_el"]["av"]["active"] = True

    if data_model["grid_el"]["av"]["active"]:
        data_model["grid_el"]["av"]["value"] = series_data["AV_GRID_EL"].to_numpy()

# PARÁMETROS DE LA RED DE GAS
data_model["grid_gas"] = {}
data_model["grid_gas"]["active"] = True
if data_model["grid_gas"]["active"]:
    
    data_model["grid_gas"]["pmax_buy"] = 300
    
    data_model["grid_gas"]["buy_price"] = {}
    data_model["grid_gas"]["buy_price"]["type"] = "variable"
    data_model["grid_gas"]["buy_price"]["value"] = series_data["C_BUY_GAS"].to_numpy()

    data_model["grid_gas"]["av"] = {}
    data_model["grid_gas"]["av"]["active"] = True

    if data_model["grid_gas"]["av"]["active"]:
        data_model["grid_gas"]["av"]["value"] = series_data["AV_GRID_GAS"].to_numpy()



# PARÁMETROS DEL GENERADOR DIESEL
#https://cdn.ade-power.com/assets/pdf/generators/cat/cat-de200gc-data-sheet.pdf
data_model["generator"] = {}
data_model["generator"]["active"] = False
if data_model["generator"]["active"]:
    data_model["generator"]["fuel_cost"] = 0.59
    data_model["generator"]["gen_cost"] = 14000
    data_model["generator"]["pmax"] = 100
    data_model["generator"]["fmin"] = 1.45
    data_model["generator"]["fmax"] = 50.8 
    data_model["generator"]["fm"] = 0.282
    data_model["generator"]["gen_OM_cost"] = 0.03
    data_model["generator"]["min_p_load"] = 20

    data_model["generator"]["av"] = {}
    data_model["generator"]["av"]["active"] = True
    if data_model["generator"]["av"]["active"]:
        data_model["generator"]["av"]["value"] = series_data["AV_DIESEL"].to_numpy()

# PARÁMETROS DEL ÁREA DISPONIBLE
data_model["area"] = {}
data_model["area"]["active"] = False
if data_model["area"]["active"]:
    data_model["area"]["value"] = 20


# PARÁMETROS LÍMITE DE INVERSIÓN
data_model["max_invest"] = {}
data_model["max_invest"]["active"] = False  
#data_model["max_invest"]["value"] = 

# PARÁMETROS DE BONOS DE CARBONO
data_model["environment"] = {}
data_model["environment"]["active"] = True
data_model["environment"]["mu"] = 266.76
data_model["environment"]["Cbono"] = 4.23

## Resolver modelo determinista

### Crear Maestro optimizacion

In [8]:
maestro_opt_estocastico = Maestro_Optimizacion(
    data_model = data_model,
    location = location,
    df_meteo = df_meteo,
    info_meteo = info,
    estocastico = False 
)

print("Escenarios:", maestro_opt_estocastico.escenarios)

Escenarios: ['main_main']


### Resolver modelo determinista

In [ ]:
maestro_opt_estocastico.LS_optimizacion()

[    9.05] Initializing SPBase
Current Iteration: 1 Time Elapsed:    0.00 Current Objective: -Inf
Current Iteration: 2 Time Elapsed:  184.69 Time Spent on Last Master:    0.24 Time Spent Generating Last Cut Set:  184.44 Current Objective:    0.00
Current Iteration: 3 Time Elapsed:  395.36 Time Spent on Last Master:    0.26 Time Spent Generating Last Cut Set:  210.41 Current Objective: 77259.61
Current Iteration: 4 Time Elapsed:  626.71 Time Spent on Last Master:    0.51 Time Spent Generating Last Cut Set:  230.84 Current Objective: 575922.14
Current Iteration: 5 Time Elapsed:  860.32 Time Spent on Last Master:    0.47 Time Spent Generating Last Cut Set:  233.13 Current Objective: 1086982.11
Current Iteration: 6 Time Elapsed: 1145.64 Time Spent on Last Master:    0.37 Time Spent Generating Last Cut Set:  284.94 Current Objective: 1241532.32
Current Iteration: 7 Time Elapsed: 1433.80 Time Spent on Last Master:    0.29 Time Spent Generating Last Cut Set:  287.88 Current Objective: 1459661

In [ ]:
variables = maestro_opt_estocastico.ls.gather_var_values_to_rank0()
for ((scen_name, var_name), var_value) in variables.items():
    print(scen_name, var_name, var_value)

main_main X_PV[Risen-590W,HYD-20kTL] 209.0
main_main X_PVs[Risen-590W,HYD-20kTL] 9.0
main_main X_PT[Termoflat 250] 28.0
main_main X_PT[VacuumTube Elite 300] 0.0
main_main X_AT[AT por sales fundidas - 5 kWh] 3.0
main_main X_AT[Tanque agua presurizada - 5kWh] 2.0
main_main X_B[PylonTech-UP5000,HYD-20kTL] 64.0
main_main X_Bs[PylonTech-UP5000,HYD-20kTL] 8.0
main_main X_CH[Risen-590W,PylonTech-UP5000,HYD-20kTL] 7.0
main_main Y_CH[Risen-590W,PylonTech-UP5000,HYD-20kTL] 1.0
main_main X_WT[FX-20kW] 0.0
main_main X_WT[TUGE-10kW] 13.0
main_main X_BOI[Type 1 - 10kWth] 1.0
main_main X_BOI[Type 1 - 5kWth] 0.0
main_main X_EH[CompactHeat 500] 0.0
main_main X_EH[PowerHeat 3000] 2.0
main_main X_CHP[Microturbina de Gas] 0.0
main_main X_CHP[Motor Reciprocante de Gas Natural] 0.0
main_main X_AC[Compacto 1 etapas - 20 kW] 0.0
main_main X_AC[Compacto 2 etapas - 30 kW] 0.0
main_main X_EC[CoolCompact 10] 0.0
main_main X_EC[PowerCool 50] 1.0
main_main Y_NEEDS['Almacenamiento de productos terminados (E: Refrige

## Resolver modelo esocástico

### Crear Maestro optimizacion

In [ ]:
maestro_opt_estocastico = Maestro_Optimizacion(
    data_model = data_model,
    location = location,
    df_meteo = df_meteo,
    info_meteo = info,
    estocastico = True  
)

print("Escenarios:", maestro_opt_estocastico.escenarios)


Escenarios: ['bad_bad', 'bad_mean', 'bad_good', 'mean_bad', 'mean_mean', 'mean_good', 'good_bad', 'good_mean', 'good_good']


### Resolver modelo estocástico

In [ ]:
maestro_opt_estocastico.LS_optimizacion()

{'root_solver': 'cplex', 'sp_solver': 'cplex', 'tol': 1e-06, 'sp_solver_options': {'threads': 16, 'parallel': -1, 'workmem': 16384, 'mip_tolerances_mipgap': 0.02, 'mip_tolerances_absmipgap': 2.0, 'mip_strategy_rinsheur': 10, 'mip_cuts_mircut': 2, 'mip_cuts_gomory': 2, 'mip_cuts_flowcovers': 2, 'mip_strategy_startalgorithm': 3, 'mip_strategy_bbinterval': 5, 'mip_display': 2}, 'max_iter': 300, 'valid_eta_lb': {'bad_bad': 0.0, 'bad_mean': 0.0, 'bad_good': 0.0, 'mean_bad': 0.0, 'mean_mean': 0.0, 'mean_good': 0.0, 'good_bad': 0.0, 'good_mean': 0.0, 'good_good': 0.0}, 'verbose': True}
[   28.35] Initializing SPBase
Current Iteration: 1 Time Elapsed:    0.00 Current Objective: -Inf
Current Iteration: 2 Time Elapsed: 2096.82 Time Spent on Last Master:    0.62 Time Spent Generating Last Cut Set: 2096.19 Current Objective:    0.00
Current Iteration: 3 Time Elapsed: 4070.18 Time Spent on Last Master:    0.39 Time Spent Generating Last Cut Set: 1972.97 Current Objective: 921396.18
Current Iteratio

In [ ]:
variables = maestro_opt_estocastico.ls.gather_var_values_to_rank0()
for ((scen_name, var_name), var_value) in variables.items():
    print(scen_name, var_name, var_value)

bad_bad X_PV[Risen-590W,HYD-20kTL] 158.0
bad_bad X_PVs[Risen-590W,HYD-20kTL] 7.0
bad_bad X_PT[Termoflat 250] 36.0
bad_bad X_PT[VacuumTube Elite 300] 0.0
bad_bad X_AT[AT por sales fundidas - 5 kWh] 3.0
bad_bad X_AT[Tanque agua presurizada - 5kWh] 1.0
bad_bad X_B[PylonTech-UP5000,HYD-20kTL] 64.0
bad_bad X_Bs[PylonTech-UP5000,HYD-20kTL] 8.0
bad_bad X_CH[Risen-590W,PylonTech-UP5000,HYD-20kTL] 7.0
bad_bad Y_CH[Risen-590W,PylonTech-UP5000,HYD-20kTL] 1.0
bad_bad X_WT[FX-20kW] 0.0
bad_bad X_WT[TUGE-10kW] 13.0
bad_bad X_BOI[Type 1 - 10kWth] 1.0
bad_bad X_BOI[Type 1 - 5kWth] 0.0
bad_bad X_EH[CompactHeat 500] 0.0
bad_bad X_EH[PowerHeat 3000] 2.0
bad_bad X_CHP[Microturbina de Gas] 0.0
bad_bad X_CHP[Motor Reciprocante de Gas Natural] 0.0
bad_bad X_AC[Compacto 1 etapas - 20 kW] 0.0
bad_bad X_AC[Compacto 2 etapas - 30 kW] 0.0
bad_bad X_EC[CoolCompact 10] 0.0
bad_bad X_EC[PowerCool 50] 0.9999999999999997
bad_bad Y_NEEDS['Almacenamiento de productos terminados (E: Refrigerador electrico, R: Sistema de 